In [17]:
#%pip install requests beautifulsoup4 edgartools lxml

logic modification 4-modify modification 1 to incorporate p formats together with div- find the level where table is and then check with PART I for the corresponding level to check if it is p or div, and then proceed

In [33]:
current_html = '../【全部24家公司】10K(01-23年)/SYK.N/SYK.N_10K_2023-12-31'

In [34]:
from lxml import etree
import re

def parse_html_content(html_content):
    parser = etree.HTMLParser(recover=True)
    return etree.fromstring(html_content, parser)

def get_xpath_of_element(element):
    """Generate XPath of the given element."""
    components = []
    
    while element is not None:
        tag = element.tag
        parent = element.getparent()
        
        if parent is not None:
            siblings = [e for e in parent.iterchildren() if e.tag == tag]
            index = siblings.index(element) + 1
            if len(siblings) > 1:
                components.append(f'{tag}[{index}]')
            else:
                components.append(tag)
        else:
            components.append(tag)
        
        element = parent

    components.reverse()
    return '/' + '/'.join(components)

def transfer_ix_elements(root):
    """
    Continuously check and move non-`ix:` child elements from `ix:` elements into their nearest non-`ix:` parent elements,
    starting from the innermost `ix:` elements and moving upwards, ensuring proper handling of nested and parallel `ix:` elements.
    """
    while True:
        if_ept = 0  # Flag to check if we moved any content

        # Iterate through all elements whose tag starts with "ix:", ensuring we process innermost ones first
        for elem in reversed(root.xpath('//*[contains(name(), "ix:")]')):  # Select all ix elements
            parent = elem.getparent()

            # Check if we should process the current element
            if elem.tag.startswith("ix:"):
                # Print the element and its current index for debugging
                #print(f"Processing element: <{elem.tag}> with text '{elem.text.strip() if elem.text else 'None'}'")

                # Move the text content from the ix: element to its parent (if any)
                if parent is not None:
                    # If there's text in the current ix: element, insert it into the parent's content
                    if elem.text and elem.text.strip():
                        text_node = etree.Element("text")  # Create a temporary element to hold the text
                        text_node.text = elem.text.strip()
                        parent.insert(parent.index(elem), text_node)  # Insert the text node at the same index as elem
                        #print(f"  Inserted text: '{elem.text.strip()}' at index {parent.index(elem)} in parent")

                    # Handle the tail content of the current ix: element (if any)
                    if elem.tail and elem.tail.strip():
                        text_node = etree.Element("text")  # Create another temporary element for tail text
                        text_node.text = elem.tail.strip()
                        parent.insert(parent.index(elem) + 1, text_node)  # Insert after the ix: element
                        #print(f"  Inserted tail: '{elem.tail.strip()}' at index {parent.index(elem) + 1} in parent")

                    # Move all non-ix child elements to the parent
                    for child in list(elem):
                        if not child.tag.startswith("ix:"):
                            # Insert before the next sibling of the element
                            parent.insert(parent.index(elem), child)
                            #print(f"  Moving child element: <{child.tag}> with text '{child.text.strip() if child.text else 'None'}'")

                    # After moving content, remove the ix: element itself
                    parent.remove(elem)
                    #print(f"  Removed element: <{elem.tag}>")
                    #print('parent', parent)

                # Mark that content was moved
                if_ept = 1

        # If no more 'ix:' elements were found, break the loop
        if if_ept == 0:
            #print("No more 'ix:' elements to move, breaking loop.")
            break

def transfer_page_elements(root):
    """
    Continuously check and move contents from `page` elements to their parent elements until all `page` elements are removed.
    """
    while True:
        if_ept = 0
        for elem in root.xpath('//page'):
            parent = elem.getparent()
            if parent is not None:
                for child in list(elem):
                    parent.insert(parent.index(elem), child)
                    if_ept = 1
                parent.remove(elem)
        if if_ept == 0:
            break

def process_all_elements(root):
    """
    Main function: move content first, then remove elements.
    """
    transfer_ix_elements(root)  # Move non `ix:` elements
    transfer_page_elements(root)

def find_toc(root):
    # XPath to find potential TOC sections, excluding those inside table elements
    toc_section_candidates = root.xpath("//span[not(ancestor::table)]|"
                                        "//p[not(ancestor::table)]|"
                                        "//font[not(ancestor::table)]|"
                                        "//b[not(ancestor::table)]")
    
    # List of possible TOC text candidates in lowercase
    toc_texts = ['table of contents', 'index', 'part i', 'part 1']

    # Iterate over all the potential candidates and check text case-insensitively
    for candidate in toc_section_candidates:
        candidate_text = candidate.xpath("string()").strip().lower()
        candidate_text = re.sub(r'\s+', ' ', candidate_text.replace('\xa0', ' '))
        if any(toc_text == candidate_text for toc_text in toc_texts) and not candidate.xpath(".//a[@href]"):
            return candidate
    
    print("TOC not found.")
    return None

def merge_similar_spans(root):
    """
    Merge consecutive <span> elements with the same style into one <span> element.
    """
    for p in root.xpath('//p'):
        children = list(p)
        i = 0
        while i < len(children) - 1:
            current = children[i]
            next = children[i + 1]
            if current.tag == 'span' and next.tag == 'span' and current.attrib == next.attrib:
                current.text = (current.text or '') + (next.text or '')
                for child in next:
                    current.append(child)
                p.remove(next)
                children.pop(i + 1)  # Ensure the list is updated correctly
            else:
                i += 1

def process_html_replace_tables(root):
    table_xpath = "//table[tr[count(td) = 2]]"

    if toc_section is None:
        return etree.tostring(root, pretty_print=True, encoding="unicode", method="html")  # No TOC, return original

    toc_position = toc_section.getparent().index(toc_section)  # Get position of TOC

    # Iterate over the found tables
    for table in root.xpath(table_xpath):
        table_position = table.getparent().index(table)
        rows = table.xpath(".//tr")
        # Ensure we are processing tables after TOC
        if table_position <= toc_position or len(rows) >5: #added or statement to rule out miscapture of item 8 in isrg 23
            continue
        #print(etree.tostring(table, pretty_print=True, encoding="unicode", method="html"))
        # Determine the type of new element based on TOC section type
        new_element = etree.Element(toc_section.tag) if toc_section.tag in ["div", "p"] else etree.Element("div")

        # Targeting the second row's <td> elements (if they exist)
        if len(rows) > 1:
            item_td = rows[1].xpath("./td[1]")
            title_td = rows[1].xpath("./td[2]")
            next_item_td = rows[2].xpath("./td[1]") if len(rows) > 2 else None
            next_title_td = rows[2].xpath("./td[2]") if len(rows) > 2 else None
        else:
            # Fallback to the first row if only one row exists
            item_td = rows[0].xpath("./td[1]")
            title_td = rows[0].xpath("./td[2]")
            next_item_td = None
            next_title_td = None

        # Function to check if a <td> is effectively empty
        def is_td_empty(td):
            return not (td.text and td.text.strip()) and len(td.getchildren()) == 0

        # Check the content of item_td and title_td
        if item_td and title_td:
            if not is_td_empty(item_td[0]) and not is_td_empty(title_td[0]):
                # Clone the contents of the <td> elements to preserve formatting
                for child in item_td[0].getchildren():
                    new_element.append(child)  # Append children of item <td>
                for child in title_td[0].getchildren():
                    new_element.append(child)  # Append children of title <td>
            elif next_item_td and next_title_td:  # Check the next row if the current one is empty
                if not is_td_empty(next_item_td[0]) and not is_td_empty(next_title_td[0]):
                    for child in next_item_td[0].getchildren():
                        new_element.append(child)  # Append children of next item <td>
                    for child in next_title_td[0].getchildren():
                        new_element.append(child)  # Append children of next title <td>
            else:
                # Print a message if no valid <td> elements were found
                table_content = etree.tostring(table, pretty_print=True, encoding="unicode", method="html")
                print(f"No valid <td> elements found in the targeted row of the table. Table content:\n{table_content}")
                continue

        # Replace the table with the new element
        table.getparent().replace(table, new_element)

    # Convert the tree back to a string, ensuring proper formatting
    return etree.tostring(root, pretty_print=True, encoding="unicode", method="html")

# Read and parse the HTML file content
with open(f'{current_html}.html', 'rb') as f:
    html_content = f.read()
root = parse_html_content(html_content)
toc_section = find_toc(root)
toc_xpath = get_xpath_of_element(toc_section)
print(toc_xpath)
# Process the parsed HTML content
process_all_elements(root)
merge_similar_spans(root)
processed_html_content = process_html_replace_tables(root)

# Generate the processed HTML string
result_html = etree.tostring(root, pretty_print=True, encoding="unicode", method="html")


/html/body/div[39]/span


In [35]:
with open('output.html', 'w', encoding='utf-8') as f:
    f.write(result_html)

print(f"Processed HTML has been saved to output.html")

Processed HTML has been saved to output.html


In [36]:
from lxml import etree
import re
import openai
import pandas as pd
from docx import Document
import json
import unicodedata

def parse_annual_report(html_content):
    parser = etree.HTMLParser(recover=True)
    tree = etree.fromstring(html_content, parser)

    def extract_index_from_xpath(xpath):
        indices = re.findall(r'\[(\d+)\]', xpath)
        return [int(index) for index in indices]

    def compare_xpaths(xpath1, xpath2):
        indices1 = extract_index_from_xpath(xpath1)
        indices2 = extract_index_from_xpath(xpath2)
        return indices1 >= indices2

    def is_within_bounds(start_xpath, end_xpath, target_xpath):
        start_index = extract_index_from_xpath(start_xpath)
        end_index = extract_index_from_xpath(end_xpath)
        target_index = extract_index_from_xpath(target_xpath)
        return start_index <= target_index <= end_index

    def is_after_start(start_xpath, target_xpath):
        start_index = extract_index_from_xpath(start_xpath)
        target_index = extract_index_from_xpath(target_xpath)
        return start_index <= target_index

    segment_dict = {}
    part_pattern = re.compile(r'^PART\s+([IVXLCDM]+|\d+)\.?', re.IGNORECASE)
    item_pattern = re.compile(r'(?i)^Item\s+\d+[A-Z]?\.*')

    def is_meaningful_text(text):
        """
        Check if the text is meaningful.
        - Must contain more than just punctuation or whitespace
        - Length check is still applied
        """
        # Remove leading/trailing whitespace
        text = text.strip()        
        # Check if text has more than just punctuation and whitespace
        return bool(re.search(r'\S', text))
    
    # Sample XPath query to get bold items
    bold_items = tree.xpath("//b | //strong | //span[contains(@style, 'font-weight:700')or contains(@style, 'font-weight:bold')] | //font[contains(@style, 'font-weight:bold')] | //p[contains(@style, 'font-weight:bold')]")
    #print(bold_items)
    # Preprocess bold items
    filtered_bold_items = []
    
    for item in bold_items:
        text_ori = ''.join(item.xpath('.//text()')).strip()
        text = unicodedata.normalize('NFKD', text_ori)
        if is_meaningful_text(text) and not item.xpath("ancestor::table"):
            filtered_bold_items.append(item)
    #print('filter check')
    #for item in filtered_bold_items:
    #    print(''.join(item.xpath('.//text()')).strip())
    #    print(get_xpath_of_element(item))
    # Initialize lists for bold items
    all_bold_items = []
    item_bold_items = []
    item_section_paths = {}

    last_item_xpath = None
    last_item_text = ""

    for item in filtered_bold_items:
        if item.tag in ['b', 'strong', 'span', 'font', 'p']:
            text_content = ''.join(item.xpath("normalize-space(string(.))")).strip().replace(u'\xa0', ' ')
            if text_content:
                xpath = get_xpath_of_element(item)
                # Get grandparent (parent of parent) to check if subtopics should be merged
                current_grandparent = item.getparent().getparent()
                last_grandparent = last_item_grandparent if 'last_item_grandparent' in locals() else None

                # Check if the current item and the last item share the same grandparent
                if last_item_text is not None and last_grandparent in ['div','p']  and current_grandparent == last_grandparent:
                    # Merge with the previous item (subtopics share the same grandparent)
                    last_item_text += f"{text_content}"
                else:
                    # Save the last item (merged or single) before moving to a new grandparent
                    if last_item_text:
                        if item_pattern.match(last_item_text):
                            item_bold_items.append({
                                'text': last_item_text,
                                'xpath': last_item_xpath
                            })
                        else:
                            if not item.xpath("ancestor::table"):
                                all_bold_items.append({
                                    'text': last_item_text,
                                    'xpath': last_item_xpath
                                })
                    
                    # Update last item info to the current one
                    last_item_text = text_content
                    last_item_xpath = xpath
                    last_item_grandparent = current_grandparent  # Track grandparent for the next iteration

                # Check if the current item matches a TOC part or item
                if compare_xpaths(xpath, toc_xpath):
                    if part_pattern.match(text_content):
                        if not item.xpath("ancestor::table"):
                            part_number = text_content
                            if part_number not in segment_dict:
                                segment_dict[part_number] = {'xpath': xpath, 'Items': {}}
                    elif item_pattern.match(text_content):
                        item_number = text_content
                        item_section_paths[item_number] = xpath

    # Ensure the last item gets processed after the loop
    if last_item_text:
        if item_pattern.match(last_item_text):
            item_bold_items.append({
                'text': last_item_text,
                'xpath': last_item_xpath
            })
        else:
            all_bold_items.append({
                'text': last_item_text,
                'xpath': last_item_xpath
            })

    print("bold items:")
    print(all_bold_items)
    print("items:")
    print(item_bold_items)

    sorted_item_paths = sorted(item_section_paths.items(), key=lambda x: extract_index_from_xpath(x[1]))

    parts = list(segment_dict.keys())
    parts.sort(key=lambda p: extract_index_from_xpath(segment_dict[p]['xpath']))
    print("dict to check")
    print(parts)
    for i, part_number in enumerate(parts):
        part_info = segment_dict[part_number]
        part_start_xpath = part_info['xpath']
        
        if i + 1 < len(parts):
            next_part_xpath = segment_dict[parts[i + 1]]['xpath']
        else:
            next_part_xpath = '/html/body'
        
        for item in item_bold_items:
            item_xpath = item['xpath']
            if is_within_bounds(part_start_xpath, next_part_xpath, item_xpath):
                segment_dict[part_number]['Items'][item['text']] = {'xpath': item_xpath, 'Subtopics': {}}
            last_part_xpath = segment_dict[parts[-1]]['xpath'] if parts else '/html/body'
            if is_after_start(last_part_xpath, item_xpath):
                segment_dict[parts[-1]]['Items'][item['text']] = {'xpath': item_xpath, 'Subtopics': {}}
    
    # Update to append subtopics correctly
    for item_number, item_info in segment_dict.items():
        if 'Items' in item_info:
            item_xpath = item_info['xpath']
            next_item_xpath = '/html/body'  # Default to the end of the document

            # Find the next item xpath
            for next_item, next_item_xpath_candidate in sorted_item_paths:
                if extract_index_from_xpath(next_item_xpath_candidate) > extract_index_from_xpath(item_xpath):
                    next_item_xpath = next_item_xpath_candidate
                    break

            current_subtopic_text = ""
            current_subtopic_xpath = None

            for subtopic in bold_items:
                if not subtopic.xpath("ancestor::table"):
                    subtopic_text_ori = ''.join(subtopic.xpath(".//text()")).strip()
                    subtopic_text = unicodedata.normalize('NFKD', subtopic_text_ori)

                    if subtopic_text:
                        subtopic_xpath = get_xpath_of_element(subtopic)

                        if is_within_bounds(item_xpath, next_item_xpath, subtopic_xpath):
                            current_index = extract_index_from_xpath(subtopic_xpath)[0]

                            # If current subtopic and the previous subtopic share the same div number, append the text
                            if current_subtopic_xpath and extract_index_from_xpath(current_subtopic_xpath)[0] == current_index:
                                current_subtopic_text += f" {subtopic_text}"
                            else:
                                # Save the previous subtopic (if exists) before starting a new one
                                if current_subtopic_text:
                                    for part_number, part_info in segment_dict.items():
                                        if item_number in part_info['Items']:
                                            item_info = part_info['Items'][item_number]
                                            if 'Subtopics' not in item_info:
                                                item_info['Subtopics'] = {}
                                            if current_subtopic_text not in item_info['Subtopics']:
                                                item_info['Subtopics'][current_subtopic_text] = {
                                                    'xpath': current_subtopic_xpath,
                                                    'content': []
                                                }
                                            item_info['Subtopics'][current_subtopic_text]['content'].append(current_subtopic_text)

                                # Start a new subtopic
                                current_subtopic_text = subtopic_text
                                current_subtopic_xpath = subtopic_xpath

            # Save the last subtopic after finishing the loop
            if current_subtopic_text:
                for part_number, part_info in segment_dict.items():
                    if item_number in part_info['Items']:
                        item_info = part_info['Items'][item_number]
                        if 'Subtopics' not in item_info:
                            item_info['Subtopics'] = {}
                        if current_subtopic_text not in item_info['Subtopics']:
                            item_info['Subtopics'][current_subtopic_text] = {
                                'xpath': current_subtopic_xpath,
                                'content': []
                            }
                        item_info['Subtopics'][current_subtopic_text]['content'].append(current_subtopic_text)

    # Ensure each item has a subtopic with the same name as the item
    for part_number, part_info in segment_dict.items():
        for item_number in part_info['Items']:
            item_info = part_info['Items'][item_number]
            if 'Subtopics' not in item_info:
                item_info['Subtopics'] = {}
            if item_number not in item_info['Subtopics']:
                item_info['Subtopics'][item_number] = {
                    'xpath': item_info['xpath'],
                    'content': []
                }
                
    # Special Items handling
    special_keywords = ['Item 1', 'Item 1A', 'Item 7', 'ITEM 1', 'ITEM 1A', 'ITEM 7']
    print(special_keywords)
    print(segment_dict)
    for part_number, part_info in segment_dict.items():
        for item_number, item_info in part_info['Items'].items():
            if any(keyword in item_number for keyword in special_keywords):
                item_xpath = item_info['xpath']
                next_item_xpath = '/html/body'
                
                # Find the next item xpath
                for next_item, next_item_xpath_candidate in sorted_item_paths:
                    if extract_index_from_xpath(next_item_xpath_candidate) > extract_index_from_xpath(item_xpath):
                        next_item_xpath = next_item_xpath_candidate
                        break

                current_subtopic_text = ""
                current_subtopic_xpath = None

                for subtopic in all_bold_items:
                    subtopic_text = subtopic['text']
                    subtopic_xpath = subtopic['xpath']
                    #print("subtopic text check")
                    #print(subtopic_text) #fine here

                    if is_within_bounds(item_xpath, next_item_xpath, subtopic_xpath):
                        #print(item_xpath, next_item_xpath, subtopic_xpath)
                        current_index = extract_index_from_xpath(subtopic_xpath)[0]
                        # If the current subtopic and the previous subtopic share the same div number, append the text
                        if current_subtopic_xpath and extract_index_from_xpath(current_subtopic_xpath)[0] == current_index:
                            current_subtopic_text += f" {subtopic_text}"
                        else:
                            if current_subtopic_text:
                                #print(current_subtopic_text)
                                if 'Subtopics' not in item_info:
                                    item_info['Subtopics'] = {}
                                if current_subtopic_text not in item_info['Subtopics']:
                                    item_info['Subtopics'][current_subtopic_text] = {
                                        'xpath': current_subtopic_xpath,
                                        'content': [current_subtopic_text]
                                    }
                                else:
                                    item_info['Subtopics'][current_subtopic_text]['content'].append(current_subtopic_text)

                            # Start a new subtopic
                            current_subtopic_text = subtopic_text
                            current_subtopic_xpath = subtopic_xpath
                    #else:
                        #print("not in boud")
                        #print(f"{subtopic_text}:{item_xpath}, {next_item_xpath}, {subtopic_xpath}")

                # Save the last subtopic after finishing the loop
                if current_subtopic_text:
                    if 'Subtopics' not in item_info:
                        item_info['Subtopics'] = {}
                    if current_subtopic_text not in item_info['Subtopics']:
                        item_info['Subtopics'][current_subtopic_text] = {
                            'xpath': current_subtopic_xpath,
                            'content': [current_subtopic_text]
                        }
                    else:
                        item_info['Subtopics'][current_subtopic_text]['content'].append(current_subtopic_text)
    print("segment dict:")
    print(segment_dict)


    def extract_content_between(start_element, end_element):
        content = {
            "tables": [],
            "text": []
        }
        def get_parent(element):
            """Retrieve the correct parent based on tag type."""
            parent_node = element.getparent()
            outer_parent_node = parent_node.getparent() if parent_node is not None else None

            # Return outer parent if element is within bold/strong tags
            if outer_parent_node is not None and parent_node.tag not in ['p', 'div'] and element.tag in ['b', 'strong']:
                return outer_parent_node
            # Return parent if it's not a div
            elif parent_node is None or parent_node.tag != 'div':
                return parent_node
            # Return outer div if both parent and outer parent are divs
            if outer_parent_node is not None and outer_parent_node.tag == 'div':
                return outer_parent_node
            # Otherwise, return the immediate parent div
            return parent_node
    
        def find_table(element):
            """Recursively search for a <table> element within children."""
            if element.tag == 'table':
                return element
            for child in element:
                result = find_table(child)  # Recursive search in child elements
                if result is not None:
                    return result
            return None  # Return None if no <table> found
        
        def parse_html_table_lxml(table_element):
            """Parse a <table> element into a structured format, adjusting for the smallest colspan and removing empty rows."""
            table_data = []
            
            # First, find the smallest colspan value in the entire table
            min_colspan = float('inf')  # Set to infinity to ensure any colspan is smaller
            for row in table_element.xpath(".//tr"):
                for cell in row.xpath(".//td | .//th"):
                    colspan = int(cell.get('colspan', 1))  # Default colspan is 1 if not specified
                    #print(colspan)
                    min_colspan = min(min_colspan, colspan)
            
            # Now, parse the table and adjust based on the smallest colspan
            for row in table_element.xpath(".//tr"):
                row_data = []
                current_column = 0  # Track the current column index in the row
                
                for cell in row.xpath(".//td | .//th"):
                    cell_text = ''.join(cell.itertext()).strip()
                    colspan = int(cell.get('colspan', 1))  # Default colspan is 1 if not specified
                    
                    # Calculate the number of empty cells needed to align this cell with the smallest colspan
                    gap = (colspan - min_colspan)  # Add this many empty cells before the actual cell
                    
                    # Add the necessary empty cells before the content
                    row_data.extend([''] * gap)
                    
                    # Add the content of the cell
                    row_data.append(cell_text)
                    current_column += colspan  # Move the column index by the colspan value
                
                # Only add rows that are not completely empty
                if any(cell != '' for cell in row_data):
                    table_data.append(row_data)
            
            return table_data
        # Identify parent nodes for traversal
        start_parent = get_parent(start_element)
        end_parent = get_parent(end_element)

            # Traverse elements between start and end
        node = start_parent
        while node is not None:
            if node.tag in ['p', 'div', 'table']:
                # Extract text if <table> is absent in this node
                table = find_table(node)
                if table is None:
                    text_content = etree.tostring(node, encoding='unicode', method='text').strip()
                    if text_content:
                        content["text"].append(text_content)
                else:
                    # If a table exists, parse it and add to the content
                    structured_table = parse_html_table_lxml(table)
                    content["tables"].append(structured_table)
            
            # Stop traversal once reaching end_element's parent
            if node == end_parent:
                break

            # Move to next node or next parent level if no sibling
            next_node = node.getnext()
            if next_node is None:
                parent_node = node.getparent()
                node = parent_node.getnext() if parent_node is not None else None
            else:
                node = next_node

        # Explicitly add the end element's content
        if end_element.tag in ['p', 'div', 'table']:
            table = find_table(end_element)
            if table is None:
                text_content = etree.tostring(end_element, encoding='unicode', method='text').strip()
                if text_content:
                    content["text"].append(text_content)
            else:
                structured_table = parse_html_table_lxml(table)
                content["tables"].append(structured_table)

        return content

    def extract_subtopics(segment_dict):
        subtopics_list = []
        
        # Add Parts
        for part_number, part_info in segment_dict.items():
            subtopics_list.append((part_number, part_info['xpath']))
            
            # Add Items within Parts
            for item_number, item_info in part_info['Items'].items():
                subtopics_list.append((item_number, item_info['xpath']))
                
                # Add Subtopics within Items
                if 'Subtopics' in item_info:
                    for subtopic_text, subtopic_info in item_info['Subtopics'].items():
                        subtopics_list.append((subtopic_text, subtopic_info['xpath']))
        
        return subtopics_list

    def get_fallback_xpath(xpath):
    # Create a fallback XPath without /span
        if '/span' in xpath:
            return xpath.rsplit('/span', 1)[0]
        return xpath

    # Generate the list of subtopics and their XPaths
    subtopics_list = extract_subtopics(segment_dict)
    print(subtopics_list)
    # Iterate through the subtopics_list and extract content
    subtopic_content = []
    for idx, (subtopic_text, subtopic_xpath) in enumerate(subtopics_list):
    # Determine the next subtopic's XPath or end of document
        if idx < len(subtopics_list) - 1:
            next_subtopic_xpath = subtopics_list[idx + 1][1]
        else:
            # For the last subtopic, handle the transition to the next part if it exists
            next_part_start_xpath = None
            for part_number, part_info in segment_dict.items():
                if part_info['xpath'] == subtopic_xpath:
                    part_index = list(segment_dict.keys()).index(part_number)
                    if part_index < len(list(segment_dict.keys())) - 1:
                        next_part_number = list(segment_dict.keys())[part_index + 1]
                        next_part_start_xpath = segment_dict[next_part_number]['xpath']
                        break
            next_subtopic_xpath = next_part_start_xpath if next_part_start_xpath else '//div[last()] | //p[last()]'

        # Get the start and end elements for extraction
        start_elements = tree.xpath(subtopic_xpath)
        end_elements = tree.xpath(next_subtopic_xpath)

        if not start_elements:
            fallback_xpath = get_fallback_xpath(subtopic_xpath)
            start_elements = tree.xpath(fallback_xpath)
            print(f"  Fallback XPath: {fallback_xpath}")

        if not end_elements:
            fallback_xpath = get_fallback_xpath(next_subtopic_xpath)
            end_elements = tree.xpath(fallback_xpath)
            print(f"  Fallback Next XPath: {fallback_xpath}")

        if not start_elements:
            print(f"  Start XPath '{subtopic_text}' not found.")
        if not end_elements:
            print(f"  End XPath '{next_subtopic_xpath}' not found.")
        if start_elements and end_elements:
            start_element = start_elements[0]
            end_element = end_elements[0]
            if start_element != end_element:
                # Extract content and tables separately
                extracted_content = extract_content_between(start_element, end_element)
                text_content = extracted_content.get('text', [])
                tables_content = extracted_content.get('tables', [])

                if text_content or tables_content:
                    # Process text content individually
                    formatted_text_content = [
                        unicodedata.normalize('NFKD', text.replace('\n', ' ').replace('  ', ' ').replace('•', '\n•'))
                        for text in text_content
                    ]

                    for part_number, part_info in segment_dict.items():
                        if 'Items' in part_info:
                            for item_number, item_info in part_info['Items'].items():
                                if 'Subtopics' in item_info and subtopic_text in item_info['Subtopics']:
                                    subtopic_info = item_info['Subtopics'][subtopic_text]
                                    
                                    # Initialize 'content' structure if not already present
                                    if 'content' not in subtopic_info or not isinstance(subtopic_info['content'], dict):
                                        subtopic_info['content'] = {
                                            "text": [],
                                            "tables": []
                                        }

                                    # Append text and tables content to their respective lists
                                    subtopic_info['content']['text'].extend(formatted_text_content)
                                    subtopic_info['content']['tables'].extend(tables_content)

                                    print(f"Updated content for subtopic '{subtopic_text}'")
        print("Content for each subtopic:")
    for part_number, part_info in segment_dict.items():
        #print(f"Part: {part_number}")
        #print(f"  XPath: {part_info['xpath']}")
        for item_number, item_info in part_info['Items'].items():
            #print(f"  Item: {item_number}")
            #print(f"    XPath: {item_info['xpath']}")
            if 'Subtopics' in item_info:
                for subtopic_text, subtopic_info in item_info['Subtopics'].items():
                    print(f"Subtopic: {subtopic_text}")
                    print(f"Subtopic XPath: {subtopic_info.get('xpath', [])}")
                    print(f"Content: {subtopic_info.get('content', [])}")
    
    return segment_dict
#print(processed_html_content)
segment_dict = parse_annual_report(result_html)

bold items:
[{'text': 'UNITED STATES', 'xpath': '/html/body/div[5]/span'}, {'text': 'SECURITIES AND EXCHANGE COMMISSION', 'xpath': '/html/body/div[6]/span'}, {'text': 'FORM 10-K', 'xpath': '/html/body/div[9]/span'}, {'text': 'OR', 'xpath': '/html/body/div[12]/span'}, {'text': 'STRYKER CORPORATION', 'xpath': '/html/body/div[16]/span'}, {'text': 'Securities registered pursuant to Section 12(g) of the Act: None', 'xpath': '/html/body/div[20]/span'}, {'text': 'DOCUMENTS INCORPORATED BY REFERENCE', 'xpath': '/html/body/div[33]/span'}, {'text': 'TABLE OF CONTENTS', 'xpath': '/html/body/div[39]/span'}, {'text': 'Business Segments and Geographic Information', 'xpath': '/html/body/div[44]/div[1]/div[10]/span'}, {'text': 'MedSurg and Neurotechnology', 'xpath': '/html/body/div[44]/div[3]/div[1]/span'}, {'text': 'Orthopaedics and Spine', 'xpath': '/html/body/div[47]/div[1]/div[1]/span'}, {'text': 'Raw Materials and Inventory', 'xpath': '/html/body/div[47]/div[3]/div[1]/span'}, {'text': 'Patents an

In [22]:
'''
with open(f'{current_html}.json', 'w') as json_file:
    json.dump(segment_dict, json_file, indent=4)
'''

"\nwith open(f'{current_html}.json', 'w') as json_file:\n    json.dump(segment_dict, json_file, indent=4)\n"

In [37]:
with open('output.json', 'w') as json_file:
    json.dump(segment_dict, json_file, indent=4)